# 局所外れ値度(LOF)の理論について


## LOFとは(by chatGPT)
* 局所外れ値度（LOF）は、データポイントがその周囲の他のデータポイントに比べてどの程度外れているか（外れ値）を定量的に評価するアルゴリズム。K近傍法を基にしており、各データポイントの「近傍」の密度と、それを他の近傍と比較することで外れ値を算出する。

### LOFの特徴
* **局所性**: データ全体ではなく、近傍領域での異常性を評価するため、不均一な分布を持つデータにも適している。
* **非パラメトリック**: 特定の分布に依存しないため、様々なデータ構造に柔軟に適用可能。
* **次元性の影響**: 高次元データでは距離計算が難しくなること（次元の呪い）があり、データの前処理が重要。

### 適用可能なデータ
* **数値データ**: 距離（ユークリッド距離、マンハッタン距離など）が計算可能なデータ。
* **時系列データ**: 特徴量を抽出して適用可能。
* **クラスタ間の密度が異なるデータ**: 局所的な特性を捉えるため、密度に偏りがあるデータに有効。

## 計算ステップ

### step1. k近傍を見つける
各データポイント$(p)$に対して、その近傍に属する$(k)$個の点（k近傍）を特定する。

+ **kの選択**: $(k)$は一般的にデータセットサイズに依存し、適切な値を試行錯誤で決定する。（例：$k=10$）
+ **距離計算**: データ間の距離（通常はユークリッド距離）を計算して、近い順にソート。

<br>

### step2. 到達可能距離を計算する
各近傍点$(q)$に対して、「到達可能距離」（Reachability Distance）を計算する。

$$
\text{Reachability Distance}(p, q) = \max(\text{k-Distance}(q), \text{Distance}(p, q))
$$

+ **k-Distance**: $(q)$のk-近傍の中で最も遠い点までの距離。
+ 到達可能距離は、異常値が極端に小さい距離を持つことで密度の誤解釈されないようにするための調整。

<br>

### step3. 局所密度を計算する
各点$(p)$の局所密度（Local Reachability Density, LRD）を計算する。

$$
\text{LRD}(p) = \left( \frac{\sum_{q \in \text{k-Nearest Neighbors}(p)} \text{Reachability Distance}(p, q)}{\lvert \text{k-Nearest Neighbors}(p) \rvert} \right)^{-1}
$$

* 局所密度は、到達可能距離の逆数の平均として定義される。
* $\text{k-Nearest Neighbors}(p)$: 点$p$の周囲で、距離が最も近い$k$個の点の集合。
* ${\lvert \cdot \rvert}$: 集合のサイズを表す記号（集合の要素数、つまり含まれる点の個数を意味する。）
* ${\lvert \text{k-Nearest Neighbors}(p) \rvert}$: 点$p$の$k$-近傍に含まれる点の数を示す。この値は常に$k$に等しくなる。

<br>

### step4. LOFスコアを計算する
LOFスコアは、対象データポイント$p$の局所密度と、その近傍点の局所密度との比率で計算する。

$$
\text{LOF}(p) = \frac{\sum_{q \in \text{k-Nearest Neighbors}(p)} \frac{\text{LRD}(q)}{\text{LRD}(p)}}{\lvert \text{k-Nearest Neighbors}(p) \rvert}
$$

+ LOFスコアの解釈:
    * $\text{LOF}(p) \approx 1$: 通常の点
    * $\text{LOF}(p) > 1$: 周囲の密度に比べて点$p$の密度が低く、外れ値の可能性が高い。
    * $\text{LOF}(p) < 1$: 周囲の密度に比べて点$p$の密度が高く、外れ値でない可能性が高い。

<br>

### step5. スコアに基づく異常検知
* データ全体でLOFスコアを計算し、ある閾値を超えるデータポイントを外れ値とみなす。
* 閾値の設定はデータやアプリケーションに応じて調整する。（例： $\text{LOF}(p) > 1.5$を外れ値とするなど）



### 局所密度の式の内容
$$
\text{LRD}(p) = \left( \frac{\sum_{q \in \text{k-Nearest Neighbors}(p)} \text{Reachability Distance}(p, q)}{\lvert \text{k-Nearest Neighbors}(p) \rvert} \right)^{-1}
$$

<br>

#### ${\sum_{q \in \text{k-Nearest Neighbors}(p)} \text{Reachability Distance}(p, q)}$ について
点$p$の$k$-近傍に含まれる点$q$それぞれに対して計算した「到達可能距離(Reachability Distance)」の合計を表す。

---

##### $Reachability Distance(p, q)$の意味
「到達可能距離」は次のように定義されている：

$$
\text{Reachability Distance}(p, q) = \max(\text{k-Distance}(q), \text{Distance}(p, q))
$$

1. $\text{k-Distance}(q)\$:
    * 点$q$の$k$-近傍における再遠点との距離
    * $q$の局所密度を測るための基準となる値
    * 異常に近い点（密集度が高すぎる点）を抑えるために使う

2. $\text{Distance}(p, q)$:
    * 点$p$と点$q$の間の距離（通常はユークリッド距離）

$\text{Reachability Distance}(p, q)$は、次のように調整されている:

* $\text{k-Distance}(q)$が$\text{Distance}(p, q)$より大きい場合は、$\text{k-Distance}(q)$を優先する。
* これにより、局所的な異常点の影響を減らす。

---

##### $\sum_{q \in \text{k-Nearest Neighbors}(p)} \text{Reachability Distance}(p, q)$の意味
この項は、点$p$の近傍に存在するすべての点$q$について、それぞれの「到達可能距離」を合計したもの。

---

##### 例
* 点$p$の$k=3$-近傍が次のように定義されているとする:
    * $\text{k-Nearest Neighbors}(p) = \{q_1, q_2, q_3}\$
* 到達可能距離が次のように計算されたとする:
    * $\text{Reachability Distance}(p, q_1) = 2.5$
    * $\text{Reachability Distance}(p, q_2) = 3.0$
    * $\text{Reachability Distance}(p, q_3) = 1.8$

この時、合計は次のようになる:
$$
\sum_{q \in \text{k-Nearest Neighbors}(p)} \text{Reachability Distance}(p, q) = 2.5 + 3.0 + 1.8 = 7.3
$$

---

##### 式の役割
この合計値は、点$p$が近傍点$q$達とどの程度密接しているかを反映している。局所到達可能距離($\text{LRD}(p)$)を計算する際、これを近傍点の数で割ることで「平均的な到達可能距離」を求め、さらにその逆数を取ることで局所密度を測定する。

---

##### 重要なポイント
1. 密集した領域では値が小さい:
    * $p$の近傍が密集している場合、各$q$の$\text{Reachability Distance}$が小さいため、合計値も小さくなる。
2. 孤立した点では値が大きい:
    * $p$が孤立している場合、$\text{Reachability Distance}$が大きくなり、合計値も大きくなる。

この合計値を通じて、点$p$の密集度を間接的に測ることができる。

---
---
#### ${\lvert \text{k-Nearest Neighbors}(p) \rvert}$について
点$p$の$k$-近傍(KNN)の数を表す。

---
##### 詳しい説明
- **$\text{k-Nearest Neighbors}(p)$:** 点 $p$ の周囲で、距離が最も近い $k$ 個の点の集合。
- **$\lvert \cdot \rvert$:** 集合のサイズを表す記号（集合の要素数、つまり含まれる点の個数を意味する）。

したがって、${\lvert \text{k-Nearest Neighbors}(p) \rvert}$ は、点 $p$ の $k$-近傍に含まれる点の数を示す。この値は常に $k$ に等しくなる。

---

##### 例
もし $k = 5$ で、点 $p$ の $k$-近傍に $q_1, q_2, q_3, q_4, q_5$ の5つの点が含まれているとします。

この場合:
$$
\text{k-Nearest Neighbors}(p) = \{q_1, q_2, q_3, q_4, q_5\}
$$
$$
\lvert \text{k-Nearest Neighbors}(p) \rvert = 5
$$

---

##### 式での役割
${\lvert \text{k-Nearest Neighbors}(p) \rvert}$は、局所到達可能距離(Local reachability Density, LRD)を計算する際の平均を取るための分母として使われている。

具体的には:
$$
\frac{\sum_{q \in \text{k-Nearest Neighbors}(p)} \text{Reachability Distance}(p, q)}{\lvert \text{k-Nearest Neighbors}(p) \rvert}
$$
は、点$p$の$k$-近傍における到達可能距離の「平均値」を計算している

---

##### 注意点
- **常に $k$ と等しい:** 正常に計算されている場合、$\lvert \text{k-Nearest Neighbors}(p) \rvert = k$ となる。
- **稀な例外:** 特殊なデータセット（例: 重複データ点や同じ距離にある点が複数存在）では、厳密に $k$ 個を特定できない場合も考えられます。この場合、距離の定義やアルゴリズムの実装に依存します。

---

##### 結論
${\lvert \text{k-Nearest Neighbors}(p) \rvert}$ は、点 $p$ の近傍点の数（通常は $k$）を表しており、平均値を取るための重要な役割を果たしている。

---

#### 最後に逆数を取る意味

逆数を取ることで**局所密度**を定義している

* **到達可能距離の平均**:
$$
\frac{\sum_{q \in \text{k-Nearest Neighbors}(p)} \text{Reachability Distance}(p, q)}{\lvert \text{k-Nearest Neighbors}(p) \rvert}
$$

    * 点$p$の近傍（$k$-Nearest Neighbors）の中で、各点との到達可能距離の平均値。
    * この部分は、点$p$が近傍からどれだけ離れているかを測る**距離の尺度**。

* **逆数を取る理由**:
    * 平均距離が大きい（点が孤立している）時に密度を小さくしたい。
    * 平均距離が小さい（点が近傍に密集している）時に密度を大きくしたい。

##### **なぜ逆数を取るのか？**
逆数を取ることで、「距離」が「密度」に変換される。

* **到達可能距離が大きい（分子が大きい）場合**:
    * 点$p$は近傍から離れており、密度が小さいとみなされる。
    * 逆数を取ることで、密度は小さい値に変換される。
* **到達可能距離が小さい（分子が小さい）場合**:
    * 点$p$は近傍に密集しており、密度が大きいとみなされる。
    * 逆数を取ることで、密度は大きな値に変換される。

##### **逆数をとる直感的な意味**
密度が大きい（近傍に点が密集している）時にLRDが高くなり、密度が小さい（点が孤立している）時にLRDが低くなる。

具体的には:
1. $\text{LRD}(p) \propto \frac{1}{\text{Average Reachability Distance}}$
    * 到達可能距離の平均が大きい（疎な領域）→ 密度（LRD）は小さい。
    * 到達可能距離の平均が小さい（密な領域）→ 密度（LRD）は大きい。
2. LRDを使うことで、点$p$が「近傍にどれだけ密集しているか」を直感的に測れるようになる。

### 局所外れ値度(LOF)の式の内容
$$
\text{LOF}(p) = \frac{\sum_{q \in \text{k-Nearest Neighbors}(p)} \frac{\text{LRD}(q)}{\text{LRD}(p)}}{\lvert \text{k-Nearest Neighbors}(p) \rvert}
$$

---

#### 分子の意味
$$
\sum_{q \in \text{k-Nearest Neighbors}(p)} \frac{\text{LRD}(q)}{\text{LRD}(p)}
$$

##### 各成分の意味
* $\text{LRD}(p)$: 点$p$の「局所到達可能密度」(LRD)
    * $p$の周囲（$p$の$k$-近傍）の密度を測る値。
    * 密度が高い（近傍が密集している）ほど値が大きい。
* $\text{LRD}(q)$: 点$q$の局所到達可能密度。
    * $q$は$p$の$k$-近傍に属する点。
* $\frac{\text{LRD}(q)}{\text{LRD}(p)}$: 点$q$と点$p$の密度の比。
    * $q$の密度が$p$の密度よりも高い時、この値は1より小さくなる。
    * 逆に、$q$の密度が$p$の密度よりも低い時、この値は1より大きくなる。

##### 分子全体の意味
$$
\sum_{q \in \text{k-Nearest Neighbors}(p)} \frac{\text{LRD}(q)}{\text{LRD}(p)}
$$
* $p$の$k$-近傍にあるすべての点$q$について、密度比の合計を算出する。
* この値は、$p$の周囲の点（近傍点）が$p$自身に比べてどれだけ密集しているかを反映する。

---

#### 分母の意味
$$
\lvert \text{k-Nearest Neighbors}(p) \rvert
$$

##### 各成分の意味
* $\text{k-Nearest Neighbors}(p)$: 点$p$の$k$-近傍に属する点の集合。
* $\lvert \text{k-Nearest Neighbors}(p) \rvert$: 点$p$の近傍に含まれる点の数。
    * 通常は$k$に等しい。（例: $k = 10$ なら $\lvert \text{k-Nearest Neighbors}(p) \rvert = 10$）

##### 分母全体の意味
この分母は、分子で計算した密度比の合計を平均するための値

---

#### 全体の式の意味
$$
\text{LOF}(p) = \frac{\sum_{q \in \text{k-Nearest Neighbors}(p)} \frac{\text{LRD}(q)}{\text{LRD}(p)}}{\lvert \text{k-Nearest Neighbors}(p) \rvert}
$$

##### 計算手順のイメージ
1. 点 $p$ の $k$-近傍に含まれる各点 $q$ の密度 ( $\text{LRD}(q)$ ) と点 $p$ の密度 ($\text{LRD}(p)$) の比を計算する。
2. 比をすべての $k$-近傍点 $q$ に対して合計する。
3. その合計値を $k$-近傍点の数（$\lvert \text{k-Nearest Neighbors}(p) \rvert$）で割り、平均を求める。

---

#### LOFスコアの解釈
LOFスコアは、点$p$が周囲の点と比べて異常であるかどうかを示す。

* $\text{LOF}(p) \approx 1$:
    * $p$の局所密度($\text{LRD}(p)$)が近傍点の密度($\text{LRD}(q)$)と大体同じである場合。
    * $p$は通常の点である可能性が高い。
* $\text{LOF}(p) > 1$:
    * $p$の局所密度($\text{LRD}(p)$)が近傍点の局所密度($\text{LRD}(q)$)よりも低い場合。
    * $p$は近傍点よりも密集度が低いため、外れ値（異常値）の可能性が高い。
* $\text{LOF}(p) < 1$:
    * $p$の局所密度($\text{LRD}(p))が近傍点の局所密度( $$\text{LRD}(q)$ )よりも高い場合。
    * $p$は異常点でない可能性が高い。
* 補足:
    * 密度が濃い：凝集している(外れ値でない)
    * 密度が薄い：離れている（外れ値）

---

#### まとめ
1. **分子**:
    * 点$p$の近傍点$q$の密度($\text{LRD}(q)$)と$p$の密度($\text{LRD}(p)$)の比の合計。
    * $p$の近傍がどれだけ$p$と密度が異なるかを測る。
2. **分母**:
    * $p$の近傍点の数（通常は$k$）
    * 分子で得られた比を平均化する役割。
3. **全体の意味**:
    * 点$p$が近傍点と比べてどれだけ密度が異なるかを示す指標。
    * スコアが大きい（$\text{LOF}(p) > 1$）と異常度が高い。



### LOFの解釈

#### 背景
LOF(局所外れ値度)は、点$p$の密度がその近傍の密度と比べてどれだけ異なるかを測定している。密度とは、ある点の周囲のデータポイントがどれだけ「近くに存在するか」を示す指標。

---

#### LOFスコア計算
* $\text{LOF}(p)$は、点$p$の**局所密度($\text{LRD}(p)$)**と、近傍点$q$の**局所密度($\text{LRD}(q)$)**を比較して求められる。
* 周囲の密度と比較して自分の密度が低ければ「異常（外れ値）の可能性が高い」とみなされる。

---

#### スコアの解釈
##### $\text{LOF}(p) \approx 1$: 通常の点
* **意味**: 点$p$の局所密度が近傍の局所密度とほぼ同じである場合、$p$は通常の点とみなされる。
* **理由**: 近傍の点も同程度の密度で分布しているため、$p$は周囲と調和した存在。
* **例**: クラスタの中央付近にある点や、クラスタの境界でも他のテント均等に分布している点。

##### $\text{LOF}(p) > 1$: 外れ値の可能性が高い
* **意味**: 点$p$の局所密度が近傍点の局所密度よりも低い場合、$p$は外れ値である可能性が高い。
* **理由**: 点$p$は「孤立」している、または「他の点が密集している領域から遠く離れている：と解釈される。
    * $\text{LRD}(p)$が小さい（点$p$の近傍に点が少ない）。
    * $\text{LRD}(q)$が大きい（点$p$の近傍に密集している点$q$が多い）。
* **例**: データクラスタから外れた位置にある点、あるいは密集した領域の近くにポツンと存在する点。

##### $\text{LOF}(p) < 1$: 外れ値でない可能性が高い
* **意味**: 点$p$の局所密度が近傍点の局所密度よりも高い場合、$p$は外れ値でない可能性が高い。
* **理由**:点$p$の周囲に他の点が密集しているため、点$p$は以上ではない可能性が高い。
    * $\text{LRD}(p)$が大きい（点$p$の近傍に多くの点が密集している）。
    * $\text{LRD}(q)$が小さい（点$q$の近傍に点が少ない）。
* **例**: データクラスタの中心に位置する点や、非常に密集した小さな領域内の点。

---

#### イメージ例
##### データ構造の視覚的な例
1. **通常の点（$\text{LOF}(p) \approx 1$）**
    * クラスタの中間層に存在。
    * 自分と近傍点の密度が似ている。
    * 周囲に点がまばらでも、それが均一なら外れ値とみなされない。
2. **外れ値（$\text{LOF}(p) > 1$）**
    * クラスタの外側や密集領域から孤立している点。
    * 周囲の点は非常に密集しているが、自分の密度は低い。
3. **密集点（$\text{LOF}(p) < 1$）**
    * 密度が高い領域（例：クラスタの中心付近）に存在。
    * 自分の密度が高いため、異常ではないとみなされる。

---

#### ※LOFスコアにおける注意点
* 上記では、外れ値の閾値を1にしているが、閾値は分布の上位1%や、標準偏差、または、ドメイン知識などに基づいて都度決める


## 正常データだけでLOFを求めて、そのLOFの最大値を閾値と定める方法のメリット・デメリット

### 背景
1. **正常データで閾値を定める理由**:
    * 異常データが含まれていると閾値の計算に悪影響を与える可能性があるため、正常データのみを用いることで基準が安定する。
    * 正常データの範囲を超えたスコアを持つデータを異常と判定できるように設計されている。
2. **閾値として最大値を採用する理由**:
    * 正常データから計算したLOFスコアの最大値は「正常データの中で最も密度が低い（境界付近の）」点に対応する。
    * 境界を超えたスコアを異常値と見なすのは理にかなっている。

---

### メリット
1. 簡単かつ直感的な方法:
    * 正常データからLOFを計算し、最大値を閾値とするだけなので、実装が容易で説明もしやすい。
2. 正常なデータのみを用いることで閾値が安定:
    * 異常データが混入するとスコア分布が歪む可能性があるが、正常データのみを使用することでそのリスクを回避できる。
3. モデルやタスクに特化しやすい:
    * この方法は、製造業や医療分野など、正常状態が非常に重要視される分野で特に有効。

---

### 潜在的な問題と注意点
1. 正常データに含まれる「境界データ」の影響
    * LOFスコアは「局所密度の低さ」を測定するため、正常データ内でも境界付近のデータ点が高いLOFスコアも持つことがある。
        * その結果、境界付近の正常データ点が閾値を押し上げ、異常値の検出感度が低下する可能性がある。
    * 対策:
        * 境界点を除外するか、LOFスコアの上位一定割合（例：95%分位点）を閾値にするなども検討する。
2. 正常データのサンプルのバリエーション
     * 正常データが十分に多様でない場合、正常データから計算されたLOFスコアが実際の異常検知に適さない可能性がある。
        * 例：正常データが非常に偏っていると、異常値との境界が正確に定まらない。
    * 対策:
        * 正常データを代表的な範囲で収集する。
        * データの分布や可視化を行い、正常データが適切に網羅されていることを確認する。
3. 新しいデータ（チェック対象）と正常データの分布が異なる場合
    * チェック対象のデータセットが、正常データセットと分布が異なる場合（例：時間の経過による分布変化や外部要因）、LOFスコアが信頼できない可能性がある。
        * 例：正常データが過去のデータで、チェック対象が新しいデータの場合。
    * 対策:
        * 定期的に正常データを更新し、モデルや閾値を再計算する。
        * チェック対象のデータを分析し、正常データとの違いを評価する。
4. 閾値の選び方が厳しすぎる場合
    * LOFの最大値をそのまま閾値とすると、異常値を検出する感度が低くなることがある。
        * 実際にはLOFの最大値ではなく、分位数（例えば95%分位点）を閾値にする方が有効な場合がある。
    * 対策:
        * 閾値の選択方法を試行錯誤し、固定値（最大値）以外の手法も検討する。
5. 次元の呪いの影響
    * 高次元データの場合、距離計算が不安定になり、LOFスコアの信頼性が低下する。
        * 特に正常データの中でも次元数が多いと、距離計算による誤差が生じやすい。
    * 対策:
        * PCAなどを用いて次元削減を行い、特徴量を適切に選定する。

---

### 結論
この方法は正常データに基づいて閾値を設定するシンプルで実用的なアプローチだが、次の点を考慮する必要がある。
1. 境界点やノイズが閾値に影響を与える可能性 -> 分位数を使用する。
2. 正常データが十分多様であることを確認する。
3. チェック対象データと正常データが同じ分布に従うことを確かめる。
4. 定期的な正常データの更新を行う。

### 2の正常データの多様性について
正常データの分布が偏っていること自体が常に問題になるわけではない。問題になるのは、正常データの偏りが**異常検知の判断基準を適切に形成できなくなる場合**。
以下で、どのような場合に分布の偏りが問題になるか説明。

---

#### 1. 正常データが偏っていても問題ない場合
正常データがが例えば範囲2~3に常に収まっている場合、それが**正確に正常なデータの特性を反映している**のであれば、偏りがあっても問題ない。
##### **なぜ問題ないのか？**
* LOFは局所密度を基準に異常値を検出する手法。従って、正常データが狭い範囲（例えば2~3）に偏っていたとしても、その範囲内での密度が均一であれば、LOFスコアは適切に計算される。
* この場合、LOFスコアに基づいて3より大きい値や2より小さい値を異常値と見なすことができ、適切に機能する。

---

### 2. 正常データの偏りが問題になる場合
問題が発生するのは、**正常データの分布が多様性を欠き、異常検知の基準が偏る場合**。

<br>

#### **ケース1：正常データの一部の領域が十分にカバーされていない場合**
* 正常データが範囲2\~3にあるとしても、実際には正常データの分布が部分的（例えば2.0\~2.5に集中）している場合。
* この場合、正常データのカバーしていない範囲（例えば2.6\~3.0）が、誤って異常と判定される可能性がある。
#### **例:**
* 2.6付近のデータポイントは、正常な範囲に属するはずだが、正常データが2.0〜2.5に集中しているため、局所密度が低くなり、LOFスコアが高くなって異常とご判定される。

<br>

#### **ケース2：正常データが複数のクラスタに別れている場合**
* 正常データが複数の異なる分布（クラスタ）に分かれている場合、LOFは「局所的な密度」を基準にするため、クラスタ間の影響を適切に処理できない可能性がある。
#### **例**
* 正常データが2.0〜2.5に80％集中し、残りの20％が4.5〜5.0に存在するとする。この場合、4.5〜5.0のクラスタ内ではLOFが正しく動作する一方で、全体の分布から見ると、この範囲は「異常」に見える可能性がある。

<br>

#### **ケース3：ノイズが含まれる場合**
* 正常データが狭い範囲に偏っていても、その中にノイズが含まれていると、LOFスコアが正確に計算されない場合がある。
#### **例**
* 2.0〜3.0の範囲に正常データがある場合でも、一部に外れたデータ（例えば3.5）が含まれていると、LOFスコアがその点に引っ張られて、全体的な閾値が歪む可能性がある。

---

### 3. どうして問題か？
正常データの偏りが問題になる理由は、LOFが「近傍密度」を基準にしているため。もし正常データの分布が次のような状況で偏ると、以下のような影響を及ぼす。
1. **正常範囲が適切に反映されない**:
    * 正常データが実際の分布をカバーできていないと、LOFの閾値が間違った範囲に設定される。
    * 本来正常な点が異常と誤判定される（偽陽性が増える）。
2. 異常範囲との境界が曖昧になる**:
    * 正常データが偏っていると、異常点との間の密度差が正確に捉えられなくなり、異常検知の性能が低下する。

---

### 4. どう対処すべきか？
* **(1) 正常データのカバー範囲を増やす**
    * 正常データが実際の正常な範囲を網羅するように、代表的なサンプルを収集する。
    * 特に、分布が広い場合は複数クラスタに分かれている場合、十分なデータ量が必要。
* **(2) データ分布を可視化する**
    * 正常データの分布を可視化（ヒストグラムや散布図）し、どの範囲がカバーされていないかを確認する。
* **(3) 正常データが偏っている場合の閾値調整**
    * 正常データのLOFスコアに基づいて閾値を調整する。例えば、最大値ではなく95％分位点を採用することで、境界点やノイズの影響を抑える。
* **(4) 正常データのクラスタを分ける**
    * 正常データが複数のクラスタに分かれている場合、それぞれのクラスタでLOFスコアを計算することで、クラスタ間の影響を分離する。

---

### 5. 具体例
##### **問題になる分布**
正常データが次のように偏っていると問題になる可能性がある。
* 正常範囲は2∼3 だが、データの80%が2.0∼2.2 に集中している。
* 正常データが2つの異なるクラスタ2.0∼2.5 と 4.0∼4.5 に分かれている。

##### **問題にならない分布**
* 正常範囲2∼3 に均一にデータが分布している場合は、特に問題ない。

---

### 結論
正常データの分布が偏ると問題になるケースは、正常データが以下の場合:
* 正常な範囲を網羅していない。
* 異常値との境界が曖昧になる。
* クラスタが複数あり、それぞれの密度が大きく異なる。

これを避けるためには、正常データを十分に収集し、分布を可視化して確認することが重要。また、閾値の調整やクラスタ別のLOF計算といったアプローチを用いることで、これらの問題に対処できる。


